# CoinMarketCap API Calls & Data Processing

In [1]:
import requests

def getListings(): #returns all coins, symbols, and ids as JSON
    # api-endpoint
    URL = "https://api.coinmarketcap.com/v2/listings/"
    
    # sending get request and saving the response as response object
    r = requests.get(url = URL)
    
    # extracting data in json format
    data = r.json()
    return data

In [2]:
#this code creates a zipped list of coin ids, names, and symbols - used later to convert recognized asset entity to id

listings = getListings()

ids = []
names = []
symbols = []
for item in listings['data']:
    _id = item['id']
    ids.append(_id)
    name = item['name']
    names.append(name)
    symbol = item['symbol']
    symbols.append(symbol)

zipped_listings = list(zip(ids,names,symbols))

In [3]:
#this code creates a dataframe of coin names and symbols (synonyms), to be written to a CSV and uploaded to Watson Entities

import pandas as pd
asset_entities = pd.DataFrame()
asset = ['Asset']*len(zipped_listings)

asset_entities['Entity'] = asset
asset_entities['Value'] = names
asset_entities['Synonyms'] = symbols
display(asset_entities.head(10))

,Entity,Value,Synonyms
0,Asset,Bitcoin,BTC
1,Asset,Litecoin,LTC
2,Asset,Namecoin,NMC
3,Asset,Terracoin,TRC
4,Asset,Peercoin,PPC
5,Asset,Novacoin,NVC
6,Asset,Feathercoin,FTC
7,Asset,Mincoin,MNC
8,Asset,Freicoin,FRC
9,Asset,Ixcoin,IXC


In [ ]:
#The above dataframe was written to a CSV file and uploaded to Watson Assistant Entities
#Within the Watson Assistant Service, Entities are essentially keywords recognized within user queries

# asset_entities.to_csv('asset_entities.csv',index=False)

In [4]:
def getTickerData(coin): 
    """returns data for an inputted coin e.g. Bitcoin"""
    coin = coin.capitalize() #capitalize the first letter so it matches with the list of asset entities in Watson Assistant
    for item in zipped_listings:
        if item[1] == coin:
            ticker_id = item[0] #find the correct id # for the queried coin, so it can be passed to the api endoint below
    
    URL = "https://api.coinmarketcap.com/v2/ticker/" + str(ticker_id) +"/"
    
    r = requests.get(url=URL)
    
    return r.json()

In [5]:
#getTickerData() FUNCTION TESTING
tickerData = getTickerData('Ethereum')

print(tickerData)
print('\n')

current_price = 'The current price of ' + str(tickerData['data']['name']) + ' is ' + '$' + \
                str(tickerData['data']['quotes']['USD']['price'])
current_volume = 'The 24hr volume of ' + str(tickerData['data']['name']) + ' is ' + '$' + \
                str(tickerData['data']['quotes']['USD']['volume_24h'])
print(current_price)
print(current_volume)

{'data': {'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'website_slug': 'ethereum', 'rank': 3, 'circulating_supply': 103547914.0, 'total_supply': 103547914.0, 'max_supply': None, 'quotes': {'USD': {'price': 118.080167891, 'volume_24h': 2115511804.51198, 'market_cap': 12226955121.0, 'percent_change_1h': -1.46, 'percent_change_24h': 4.74, 'percent_change_7d': 8.19}}, 'last_updated': 1543722080}, 'metadata': {'timestamp': 1543721342, 'error': None}}


The current price of Ethereum is $118.080167891
The 24hr volume of Ethereum is $2115511804.51198


In [6]:
def getGlobal(): #returns global data for crypto market (e.g. # of active currencies, market cap, dominance etc)

    URL = 'https://api.coinmarketcap.com/v2/global/'
    
    r = requests.get(url=URL)
    
    return r.json()


globalData = getGlobal()

#INITIALIZE GLOBAL VARIABLES FOR GLOBAL DATA (these values don't depend on Watson recognizing an 'Asset' in the query)
active_coins = globalData['data']['active_cryptocurrencies']
print('The # of active coins on CMC is: ' +str(active_coins))

active_markets = globalData['data']['active_markets']
print('The # of active markets (exchanges) on CMC is: ' +str(active_markets))

btc_dominance = globalData['data']['bitcoin_percentage_of_market_cap']
print("Bitcoin's current market dominance is: " +str(btc_dominance) + '%')

total_market_cap = globalData['data']['quotes']['USD']['total_market_cap']
print('The current total global market capitalization is: ' + '$' + str(total_market_cap))

total_volume_24hr = globalData['data']['quotes']['USD']['total_volume_24h']
print('The global trading volume in the last 24 hours was: ' + '$' + str(total_volume_24hr))

The # of active coins on CMC is: 2073
The # of active markets (exchanges) on CMC is: 15677
Bitcoin's current market dominance is: 53.67%
The current total global market capitalization is: $136211118874.0
The global trading volume in the last 24 hours was: $15170097096.0


# Watson API Authentication

In [7]:
from watson_developer_cloud import AssistantV1
import json
from watson_developer_cloud import WatsonApiException

assistant = AssistantV1(
    version='2018-09-20',
    username='7144b623-4797-4430-8f77-733cc1f2ab0b',
    password='AnjnaPUbKZlC',
    url='https://gateway.watsonplatform.net/assistant/api'
)

workspaces = assistant.list_workspaces().get_result() #this variable contains the workspace_id 

# Query Testing

In [8]:
#USE THIS CELL TO TEST INDIVIDUAL QUERIES OUTSIDE OF A WHILE LOOP

response = assistant.message(
    workspace_id = 'acd1683d-8510-43ca-8553-af8895bea099',
    input={
        'text': "how many eth can i buy with 2 bitcoin?" #test different queries here
    }
).get_result()


intent = response['intents'][0]['intent'] 

entities = response['entities']
asset_entity = [x for x in entities if x['entity'] == 'Asset']
if asset_entity != []:
    coin_name = asset_entity[0]['value']


print('Detected intent: ' + str(intent))
print('Detected asset: ' + str(coin_name))

# print(json.dumps(response, indent=2))

Detected intent: CompareCoins
Detected asset: Ethereum


# Main Conversation Loop

In [12]:
from decimal import Decimal
import re

user_input = 'start conversation'

while user_input != '':
    user_input = input()

    response = assistant.message(
        workspace_id = 'acd1683d-8510-43ca-8553-af8895bea099',
        input={
            'text': user_input
        }
    ).get_result()
    
    intent = response['intents'][0]['intent']
    entities = response['entities']
    asset_entity = [x for x in entities if x['entity'] == 'Asset'] #isolate 'Asset' entity when multiple entities are detected
    marketShare_entity = [x for x in entities if x['entity'] == 'MarketShare']  
    change_entity = [x for x in entities if x['entity'] == 'Change'] #change entity for 'CompareCoins' functionality
    if asset_entity != []:
        coin_name = asset_entity[0]['value'] #isolate single coin name in query

        
    if intent == 'General_Greetings':
        print('>>>' + response['output']['text'][0])
        
    elif intent == 'General_Agent_Capabilities':
        print('>>>I am a cryptocurrency chatbot that pulls real-time data from')
        print('an exchange aggregator service. You can ask me about prices, volume,') 
        print('market capitalization, price changes, coin supply, and global metrics.')
        
    elif intent == 'QuestionDataSource':
        print('>>>' + response['output']['text'][0])
        
    elif intent == 'GetQuote': #return a full quote for a specific coin
        print('>>>' + response['output']['text'][0] + coin_name + ':')
        print('CURRENT PRICE: $' + str(getTickerData(coin_name)['data']['quotes']['USD']['price']))
        print('24HR VOLUME: $' + str(getTickerData(coin_name)['data']['quotes']['USD']['volume_24h']))
        print('PERCENT CHANGE 1HR: ' + str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_1h']) + '%')
        print('PERCENT CHANGE 24HR: ' +str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_24h']) + '%')
        print('PERCENT CHANGE 7D: ' +str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_7d']) + '%')
        print('MARKET CAPITALIZATION: $' + str(getTickerData(coin_name)['data']['quotes']['USD']['market_cap']))
        print('MARKET CAP RANK: ' + str(getTickerData(coin_name)['data']['rank']))
        print('CIRCULATING SUPPLY: ' + str(getTickerData(coin_name)['data']['circulating_supply']))
        print('TOTAL SUPPLY: ' + str(getTickerData(coin_name)['data']['total_supply']))
        print('MAX SUPPLY: ' + str(getTickerData(coin_name)['data']['max_supply']))
              
              
    elif intent == 'GetPrice' and asset_entity != []:
        print('>>>' + response['output']['text'][0] + ' ' + coin_name + ' is $' + 
              str(getTickerData(coin_name)['data']['quotes']['USD']['price']))
        
    elif intent == 'GetCoinVolume' and asset_entity != []:
        print('>>>' + response['output']['text'][0] + coin_name + ' is $' + 
              str(getTickerData(coin_name)['data']['quotes']['USD']['volume_24h']))
    
    elif intent == 'GetPercentChange' and asset_entity != [] and change_entity != []:
        asset_entity = [x for x in entities if x['entity'] == 'Asset']
        if asset_entity != []:
            coin_name = asset_entity[1]['value']
        print('>>>' + response['output']['text'][0] + coin_name + ':')
        print('PERCENT CHANGE 1HR: ' + str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_1h']) + '%')
        print('PERCENT CHANGE 24HR: ' +str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_24h']) + '%')
        print('PERCENT CHANGE 7D: ' +str(getTickerData(coin_name)['data']['quotes']['USD']['percent_change_7d']) + '%')
        
    elif intent == 'GetCoinMarketCap' and asset_entity != []:
        print('>>>' + response['output']['text'][0] + coin_name + ' is $' + 
              str(getTickerData(coin_name)['data']['quotes']['USD']['market_cap']))
        
    elif intent == 'GetCoinMarketShare' and asset_entity != [] and marketShare_entity != []:
        marketShare = round(Decimal(getTickerData(coin_name)['data']['quotes']['USD']['market_cap'] / total_market_cap * 100),2)
        print('>>>' + response['output']['text'][0] + coin_name + ' is ' + str(marketShare) + '%')
        
    elif intent == 'GetGlobalVolume':
        print('>>>' + response['output']['text'][0] + '$' + str(round(Decimal(total_volume_24hr),2)))
        
    elif intent == 'GetGlobalMarketCap':
        print('>>>' + response['output']['text'][0] + '$' + str(round(Decimal(total_market_cap),2)))
        
    elif intent == 'GetBitcoinDominance':
        print('>>>' + response['output']['text'][0] + str(btc_dominance) + '%')
        
    elif intent == 'CompareCoins':
        query = response['input']['text']
        asset_entity = [x for x in entities if x['entity'] == 'Asset']
        coins = [x['value'] for x in asset_entity]
        base_coin = coins[1]
        base_coin_price = getTickerData(base_coin)['data']['quotes']['USD']['price']
        convert_coin = coins[0]
        convert_coin_price = getTickerData(convert_coin)['data']['quotes']['USD']['price']
        
        digit = re.findall(r'\d+', query)
        if digit != []:
            digit = digit[0]
            purchase_power = int(digit) * base_coin_price
            ratio = round(Decimal(purchase_power / convert_coin_price),2)
            print('>>>' + response['output']['text'][0] + str(ratio) + ' ' + convert_coin + ' with ' + str(digit) + ' ' + base_coin)
        elif 'ratio' in query:
            ratio = round(Decimal(convert_coin_price / base_coin_price),2)
            print('>>>One ' + convert_coin + ' is worth ' + str(ratio) + ' ' + base_coin)
        elif 'compare' in query:
            ratio = round(Decimal(convert_coin_price / base_coin_price),2)
            print('>>>One ' + convert_coin + ' is worth ' + str(ratio) + ' ' + base_coin)
            
            base_coin_1hr = getTickerData(base_coin)['data']['quotes']['USD']['percent_change_1h']
            base_coin_24hr = getTickerData(base_coin)['data']['quotes']['USD']['percent_change_24h']
            base_coin_7d = getTickerData(base_coin)['data']['quotes']['USD']['percent_change_7d']
            convert_coin_1hr = getTickerData(convert_coin)['data']['quotes']['USD']['percent_change_1h']
            convert_coin_24hr = getTickerData(convert_coin)['data']['quotes']['USD']['percent_change_24h']
            convert_coin_7d = getTickerData(convert_coin)['data']['quotes']['USD']['percent_change_7d']
            
            diff_1hr = abs(abs(base_coin_1hr) - abs(convert_coin_1hr)) 
            diff_24hr = abs(abs(base_coin_24hr) - abs(convert_coin_24hr))
            diff_7d = abs(abs(base_coin_7d) - abs(convert_coin_7d))
            
            if int(convert_coin_1hr) > int(base_coin_1hr):
                print('>>>' + convert_coin + ' is outperforming ' + base_coin + ' by ' + 
                      str(round(Decimal(diff_1hr),2))+'%'+' on a 1 hour basis')
            else:
                print('>>>' + base_coin + ' is outperforming ' + convert_coin + ' by ' + 
                      str(round(Decimal(diff_1hr),2))+'%'+' on a 1 hour basis')
                
            if int(convert_coin_24hr) > int(base_coin_24hr):
                print('>>>' + convert_coin + ' is outperforming ' + base_coin + ' by ' + 
                      str(round(Decimal(diff_24hr),2))+'%'+' on a 24 hour basis')
            else:
                print('>>>' + base_coin + ' is outperforming ' + convert_coin + ' by ' + 
                      str(round(Decimal(diff_24hr),2))+'%'+' on a 24 hour basis')
                
            if int(convert_coin_7d) > int(base_coin_7d):
                print('>>>' + convert_coin + ' is outperforming ' + base_coin + ' by ' + 
                      str(round(Decimal(diff_7d),2))+'%'+' on a 7 day basis')
            else:
                print('>>>' + base_coin + ' is outperforming ' + convert_coin + ' by ' + 
                      str(round(Decimal(diff_7d),2))+'%'+' on a 7 day basis')

            
    elif intent == 'General_Ending':
        print('>>>Goodbye, thanks for using the CMC bot!')
        break
        
    else:
        print('>>>Sorry, that question is out of my scope')
        
    print('\n')

hello
>>>Hello, what can I help you with?


what do you do?
>>>I am a cryptocurrency chatbot that pulls real-time data from
an exchange aggregator service. You can ask me about prices, volume,
market capitalization, price changes, coin supply, and global metrics.


where do you get your data?
>>>I source all my data from https://coinmarketcap.com/ 


what is the current price of bitcoin?
>>>The current price of Bitcoin is $4195.23662398


what is the total global market cap?
>>>The aggregate global market capitalization of all circulating cryptocurrencies is $136211118874.00


what is the global trading volume?
>>>The 24 hour global trading volume is $15170097096.00


what is the current trading volume of ethereum
>>>The 24 hour volume of Ethereum is $2116363739.47368


what is the market share of ethereum
>>>The market share of Ethereum is 8.97%


what is bitcoin's market share
>>>Bitcoin's market dominance (% of total market cap) is 53.67%


what is the percent change of monero?
>>>H